In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
forex_schema = StructType(fields=[StructField("base_currency_id", StringType(), False),
                                     StructField("target_currency_id", StringType(), False),
                                     StructField("amount", DoubleType(), True),
                                     StructField("data_time", TimestampType(), True),
                                     StructField("time_insert", TimestampType(), True),
                                    ])
df_raw_forex = spark.read.csv("/mnt/forexratesdl/raw/forex_clean.csv", sep=r',',schema=forex_schema ,header=True)

In [0]:
df_raw_lookup = spark.read.csv("/mnt/forexratesdl/lookup/forex_info.csv", sep=r',',header=True)
df_raw_lookup = df_raw_lookup.withColumnRenamed('currency name','currency_name')
df_raw_lookup = df_raw_lookup.withColumn('currency_name', regexp_replace(df_raw_lookup.currency_name,'	', ''))
df_raw_lookup = df_raw_lookup.withColumn('currency_name', regexp_replace(df_raw_lookup.currency_name,'�','o'))


In [0]:
forex_df = df_raw_lookup.join(df_raw_forex, df_raw_forex.target_currency_id == df_raw_lookup.base, "inner") \
    .select(df_raw_forex.base_currency_id,df_raw_forex.target_currency_id,df_raw_lookup.currency_name,df_raw_forex.amount,df_raw_forex.data_time)

In [0]:
forex_df.write.mode("append").format("delta").saveAsTable("forex_db.forex_data")